In [1]:
import csv
import pandas as pd
import numpy as np
import datetime
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.impute import SimpleImputer

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

data = []

data_1 = 'RESIDIV_Vimala.csv'
data_2 = 'VV_RESIDIV_Vimala.csv'

In [2]:
data_labeled = pd.read_csv(data_1)
data_unlabeled = pd.read_csv(data_2) 

In [3]:
data_unlabeled

,Pas_ID,Født,Oppdaget_når,Age,Død,lever,FIGO_stadium,Substadium,histologi,gradering,kreftform,OVAgrade,myometireinvasjon,Cx_invasjon,Karinvasjon,Lymfekarinvasjon,Lymfeknuter,Tumor_str,hist_nummer,Primærbehandling,Avsluttet_prim_beh,Ant_ktr,Beh_sykehus,Ktr_ingen,Ktr_priv,Ktr_lok,Ktr_Uni,Ktr_komb,Ktr_fastlege,Ktr_andre,Lengde_sympt_dager,Lengde_sympt_uker,Lengde_sympt_mnd,Symptomer_mindre_1mnd,TidSympt_2dic,Oppdaget_pas,Oppdaget_FL,Oppdaget_Priv,Oppdaget_Lok,Oppdaget_Univ,Oppdaget_Kir,Oppdaget_annen,Tidligere_kontakt,Sympt_tilstede,Sympt_blødning,Sympt_smerter,Sympt_vekttap,Sympt_utflod,Sympt_ascites,Sympt_avføring,Sympt_ileus,Sympt_fatigue,Sympt_vannlating,Sympt_blodfeces,Sympt_blodurin,Sympt_anorexi,Sympt_kakeksi,Sympt_hudtumor,sympt_luftveier,Sympt_annet,Verif_histo,Histo_nummer,Histo_lab,Verif_cytol,Cytol_nummer,Cytol_lab,Verif_MR,Verif_CT,Verif_RTH,Verif_ABD_UL,Verif_VAG_UL,ULverif,Verif_Colon,Verif_annet,Lok_vagina,Lok_bekken,Lok_ØvreAbd,Lok_lunger,Lok_lever,Lok_lok_LK,Lok_fjern_LK,Lok_skjelett,Lok_hjernen,Lok_annet,Henvist_Uni,Hvorfor_ikke,Behandling,Time_to_recurrence_months,symptomer_kliniske,KlinSympt_2dic,symptomer,Symtomer_tilstede,Sympt_tilstede2dic,Symptomer_lengde,Distal_recurrence,Local_recurrence,Age_at_diagnosis,age_rec
0,5013,3/22/1937,1/1/2013,75,,,1,3,1,2,1,,,2,2,,2,,,1,3/1/2009,8,5,0,0,0,5,0,0,,5,0,0,1,1,1,0,0,0,0,0,,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,,0,,,1,4156,5,0,1,0,1,1,1,0,,0,0,0,0,0,0,1,0,0,,2,konsultert HUS,2,46,1,1,5,2,1,0,1,0,71,75
1,1244,3/4/1941,12/1/2015,74,,1,1,2,1,2,1,,2,2,1,,,,1503931,1,3/1/2015,3,,0,0,0,0,,0,,0,1,0,1,1,1,0,0,1,1,0,,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,,1,15160,1,1,16000088,,0,1,0,1,1,1,0,,1,0,1,0,0,0,1,0,0,,1,,3,9,3,1,1,2,1,0,1,0,73,74
2,2116,4/27/1934,8/1/2014,80,,,1,3,1,1,1,,,,,,,,819216,5,1/1/2009,16,2,0,1,0,0,0,0,,0,0,1,0,1,1,0,1,0,1,0,,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,,1,1427589,2,0,,,0,1,0,1,1,1,0,,0,1,0,0,0,0,0,0,0,,1,,3,67,1,1,1,2,1,2,0,1,74,80
3,5017,5/21/1935,11/1/2013,78,,,1,1,6,,1,,,,,,,,4884,1,3/1/2013,2,5,0,0,0,5,0,0,,0,3,0,1,1,0,0,1,0,0,0,,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,,1,632,5,0,,,0,1,0,1,1,1,0,,0,1,1,0,0,0,0,0,0,,1,,3,8,1,1,3,2,1,0,1,0,77,78
4,5039,2/15/1936,8/1/2015,79,,,3,3,6,,1,,,,,,,,23874,5,4/1/2015,1,5,0,0,0,5,0,0,,0,0,0,0,0,0,0,0,0,1,0,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,1,17694,5,0,,,0,1,0,1,1,1,0,,1,0,0,0,0,0,0,0,0,,1,,3,4,0,0,0,0,0,1,0,1,79,79
5,11012,4/20/1932,11/1/2015,83,,1,1,2,1,2,1,,2,2,1,,,,12628,1,7/1/2014,6,11,0,1,0,0,0,0,,0,0,0,0,0,0,0,1,0,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,0,,,0,,,1,0,0,0,1,1,0,,0,1,0,0,0,0,0,0,0,,1,,,16,0,0,0,0,0,1,0,1,82,83
6,29014,8/26/1931,2/1/2014,82,,1,1,2,1,2,1,,2,2,2,3,2,,,5,11/1/2011,7,29,0,0,1,0,0,0,,0,0,0,0,0,0,0,0,1,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,0,,,1,1643,16,1,0,0,0,1,1,0,,1,0,0,0,0,0,0,0,0,,,,1,27,0,0,0,0,0,1,0,1,80,82
7,1150,4/22/1936,5/1/2014,78,,,1,2,1,1,1,,2,2,2,3,2,,1307185,1,8/1/2013,3,1,0,0,1,0,0,0,,0,0,0,0,0,0,0,0,1,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,1,12883,8,0,,,0,1,0,0,1,1,0,,1,0,0,0,0,0,0,0,0,,1,,2,9,0,0,0,0,0,1,0,1,77,78
8,1176,4/27/1934,11/1/2014,80,,,1,2,1,2,1,,2,2,1,3,3,,,1,4/1/2014,2,6,0,0,0,6,0,0,,0,0,12,0,1,1,0,0,1,0,0,,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,,1,13266,6,0,,,0,1,1,0,1,1,0,,1,0,0,0,0,0,0,0,0,,1,,2,7,1,1,12,2,1,13,0,1,79,80
9,1184,3/18/1927,12/1/2014,87,,,1,2,12,1,1,,2,2,1,,,,1111414,1,2/1/2011,10,31,0,0,1,0,0,0,,0,0,0,0,0,0,0,0,1,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,1,1500651,,0,,,0,1,0,0,1,1,0,,0,0,0,0,0,1,0,0,0,,1,,2,46,0,0,0,0,0,1,1,0,83,87


In [4]:
len(data_unlabeled)

680

In [5]:
def convert_time_to_integer(value):
    r = re.compile('\d*\/\d*\/\d*')
    if r.match(value) == None:
        pass
    else:
        month, date, year = value.split('/')
        date = pd.to_numeric(date, errors='coerce')
        month = pd.to_numeric(month, errors='coerce')
        year = pd.to_numeric(year, errors='coerce')
    
        return datetime.datetime(year, month, date).timestamp()/10**6

In [6]:
data_unlabeled['Født'] = data_unlabeled['Født'].apply(convert_time_to_integer)
data_unlabeled['Oppdaget_når'] = data_unlabeled['Oppdaget_når'].apply(convert_time_to_integer)
data_unlabeled['Avsluttet_prim_beh'] = data_unlabeled['Avsluttet_prim_beh'].apply(convert_time_to_integer)


In [7]:
def fill_nan_with_mean(df):
    return df.fillna(df.mean())

In [8]:
data_unlabeled['Født'] = fill_nan_with_mean(data_unlabeled['Født'])
data_unlabeled['Oppdaget_når'] = fill_nan_with_mean(data_unlabeled['Oppdaget_når'])
data_unlabeled['Avsluttet_prim_beh'] = fill_nan_with_mean(data_unlabeled['Avsluttet_prim_beh'])
data_unlabeled['Age'] = pd.to_numeric(data_unlabeled['Age'], errors='coerce')


In [9]:
data_unlabeled.head()

,Pas_ID,Født,Oppdaget_når,Age,Død,lever,FIGO_stadium,Substadium,histologi,gradering,kreftform,OVAgrade,myometireinvasjon,Cx_invasjon,Karinvasjon,Lymfekarinvasjon,Lymfeknuter,Tumor_str,hist_nummer,Primærbehandling,Avsluttet_prim_beh,Ant_ktr,Beh_sykehus,Ktr_ingen,Ktr_priv,Ktr_lok,Ktr_Uni,Ktr_komb,Ktr_fastlege,Ktr_andre,Lengde_sympt_dager,Lengde_sympt_uker,Lengde_sympt_mnd,Symptomer_mindre_1mnd,TidSympt_2dic,Oppdaget_pas,Oppdaget_FL,Oppdaget_Priv,Oppdaget_Lok,Oppdaget_Univ,Oppdaget_Kir,Oppdaget_annen,Tidligere_kontakt,Sympt_tilstede,Sympt_blødning,Sympt_smerter,Sympt_vekttap,Sympt_utflod,Sympt_ascites,Sympt_avføring,Sympt_ileus,Sympt_fatigue,Sympt_vannlating,Sympt_blodfeces,Sympt_blodurin,Sympt_anorexi,Sympt_kakeksi,Sympt_hudtumor,sympt_luftveier,Sympt_annet,Verif_histo,Histo_nummer,Histo_lab,Verif_cytol,Cytol_nummer,Cytol_lab,Verif_MR,Verif_CT,Verif_RTH,Verif_ABD_UL,Verif_VAG_UL,ULverif,Verif_Colon,Verif_annet,Lok_vagina,Lok_bekken,Lok_ØvreAbd,Lok_lunger,Lok_lever,Lok_lok_LK,Lok_fjern_LK,Lok_skjelett,Lok_hjernen,Lok_annet,Henvist_Uni,Hvorfor_ikke,Behandling,Time_to_recurrence_months,symptomer_kliniske,KlinSympt_2dic,symptomer,Symtomer_tilstede,Sympt_tilstede2dic,Symptomer_lengde,Distal_recurrence,Local_recurrence,Age_at_diagnosis,age_rec
0,5013,-1034.4708,1356.9948,75.0,,,1,3,1,2,1,,,2,2,,2,,,1,1235.8620,8,5,0,0,0,5,0,0,,5,0,0,1,1,1,0,0,0,0,0,,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,,0,,,1,4156,5,0,1,0,1,1,1,0,,0,0,0,0,0,0,1,0,0,,2,konsultert HUS,2,46,1,1,5,2,1,0,1,0,71,75
1,1244,-909.7992,1448.9244,74.0,,1,1,2,1,2,1,,2,2,1,,,,1503931,1,1425.1644,3,,0,0,0,0,,0,,0,1,0,1,1,1,0,0,1,1,0,,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,,1,15160,1,1,16000088,,0,1,0,1,1,1,0,,1,0,1,0,0,0,1,0,0,,1,,3,9,3,1,1,2,1,0,1,0,73,74
2,2116,-1126.0548,1406.8440,80.0,,,1,3,1,1,1,,,,,,,,819216,5,1230.7644,16,2,0,1,0,0,0,0,,0,0,1,0,1,1,0,1,0,1,0,,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,,1,1427589,2,0,,,0,1,0,1,1,1,0,,0,1,0,0,0,0,0,0,0,,1,,3,67,1,1,1,2,1,2,0,1,74,80
3,5017,-1092.4452,1383.2604,78.0,,,1,1,6,,1,,,,,,,,4884,1,1362.0924,2,5,0,0,0,5,0,0,,0,3,0,1,1,0,0,1,0,0,0,,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,,1,632,5,0,,,0,1,0,1,1,1,0,,0,1,1,0,0,0,0,0,0,,1,,3,8,1,1,3,2,1,0,1,0,77,78
4,5039,-1069.1172,1438.3800,79.0,,,3,3,6,,1,,,,,,,,23874,5,1427.8392,1,5,0,0,0,5,0,0,,0,0,0,0,0,0,0,0,0,1,0,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,1,17694,5,0,,,0,1,0,1,1,1,0,,1,0,0,0,0,0,0,0,0,,1,,3,4,0,0,0,0,0,1,0,1,79,79


In [10]:
preprocessed_df = data_unlabeled.drop(['Ktr_andre', 'Oppdaget_annen', 'Sympt_annet', 'Verif_annet', 'Lok_annet', 'Hvorfor_ikke'], axis=1)

In [11]:
df_without_alive_death = data_unlabeled.drop(['Død', 'lever', 'Ktr_andre', 'Oppdaget_annen', 'Sympt_annet', 'Verif_annet', 'Lok_annet', 'Hvorfor_ikke'], axis=1)

In [15]:
type(preprocessed_df['FIGO_stadium'][0])

numpy.int64

In [12]:
df_without_alive_death.head()

,Pas_ID,Født,Oppdaget_når,Age,FIGO_stadium,Substadium,histologi,gradering,kreftform,OVAgrade,myometireinvasjon,Cx_invasjon,Karinvasjon,Lymfekarinvasjon,Lymfeknuter,Tumor_str,hist_nummer,Primærbehandling,Avsluttet_prim_beh,Ant_ktr,Beh_sykehus,Ktr_ingen,Ktr_priv,Ktr_lok,Ktr_Uni,Ktr_komb,Ktr_fastlege,Lengde_sympt_dager,Lengde_sympt_uker,Lengde_sympt_mnd,Symptomer_mindre_1mnd,TidSympt_2dic,Oppdaget_pas,Oppdaget_FL,Oppdaget_Priv,Oppdaget_Lok,Oppdaget_Univ,Oppdaget_Kir,Tidligere_kontakt,Sympt_tilstede,Sympt_blødning,Sympt_smerter,Sympt_vekttap,Sympt_utflod,Sympt_ascites,Sympt_avføring,Sympt_ileus,Sympt_fatigue,Sympt_vannlating,Sympt_blodfeces,Sympt_blodurin,Sympt_anorexi,Sympt_kakeksi,Sympt_hudtumor,sympt_luftveier,Verif_histo,Histo_nummer,Histo_lab,Verif_cytol,Cytol_nummer,Cytol_lab,Verif_MR,Verif_CT,Verif_RTH,Verif_ABD_UL,Verif_VAG_UL,ULverif,Verif_Colon,Lok_vagina,Lok_bekken,Lok_ØvreAbd,Lok_lunger,Lok_lever,Lok_lok_LK,Lok_fjern_LK,Lok_skjelett,Lok_hjernen,Henvist_Uni,Behandling,Time_to_recurrence_months,symptomer_kliniske,KlinSympt_2dic,symptomer,Symtomer_tilstede,Sympt_tilstede2dic,Symptomer_lengde,Distal_recurrence,Local_recurrence,Age_at_diagnosis,age_rec
0,5013,-1034.4708,1356.9948,75.0,1,3,1,2,1,,,2,2,,2,,,1,1235.8620,8,5,0,0,0,5,0,0,5,0,0,1,1,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,,1,4156,5,0,1,0,1,1,1,0,0,0,0,0,0,0,1,0,0,2,2,46,1,1,5,2,1,0,1,0,71,75
1,1244,-909.7992,1448.9244,74.0,1,2,1,2,1,,2,2,1,,,,1503931,1,1425.1644,3,,0,0,0,0,,0,0,1,0,1,1,1,0,0,1,1,0,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,15160,1,1,16000088,,0,1,0,1,1,1,0,1,0,1,0,0,0,1,0,0,1,3,9,3,1,1,2,1,0,1,0,73,74
2,2116,-1126.0548,1406.8440,80.0,1,3,1,1,1,,,,,,,,819216,5,1230.7644,16,2,0,1,0,0,0,0,0,0,1,0,1,1,0,1,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1427589,2,0,,,0,1,0,1,1,1,0,0,1,0,0,0,0,0,0,0,1,3,67,1,1,1,2,1,2,0,1,74,80
3,5017,-1092.4452,1383.2604,78.0,1,1,6,,1,,,,,,,,4884,1,1362.0924,2,5,0,0,0,5,0,0,0,3,0,1,1,0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,632,5,0,,,0,1,0,1,1,1,0,0,1,1,0,0,0,0,0,0,1,3,8,1,1,3,2,1,0,1,0,77,78
4,5039,-1069.1172,1438.3800,79.0,3,3,6,,1,,,,,,,,23874,5,1427.8392,1,5,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,17694,5,0,,,0,1,0,1,1,1,0,1,0,0,0,0,0,0,0,0,1,3,4,0,0,0,0,0,1,0,1,79,79


In [13]:
%store data_unlabeled
%store data_labeled
%store preprocessed_df
%store df_without_alive_death

Stored 'data_unlabeled' (DataFrame)
Stored 'data_labeled' (DataFrame)
Stored 'preprocessed_df' (DataFrame)
Stored 'df_without_alive_death' (DataFrame)
